# Рогачев Александр Викторович, 521 группа
# Вариант 6

In [1]:
from math import *  # for math functions and constants. Trigonometry functions inputs are in radians
import numpy as np  # for matrix operations such as matrix multiplication

## Первый блок задач

## 1. 
Пусть заданы гринвичские координаты $ η_1 , η_2 , η_3 $ :
$$ 2550804.91, 2460159.42, 5285668.08 м.$$
Вычислите для этих координат значения географических и геоцентрических координат. Приведите код программы.

In [2]:
# problem variables and constants using the GLONASS model 
eta_1, eta_2, eta_3 = 2550804.91, 2460159.42, 5285668.08
a = 6378136.0  # GLONASS model
e2 = 6.69436619 * 10**(-3) 
print(e2)
eps = 1.0/298.257839303 

stop_condition = 1

# initial condition

lambd = atan2(eta_2, eta_1) 

phi0 = atan2(eta_3, sqrt(eta_1**2 + eta_2**2))
phi_0 = phi0 + 0.5 * e2 * sin(2 * phi0)
h_0 = sqrt(eta_1**2 + eta_2**2 + eta_3**2) - a + 0.5 * a * e2 * sin(phi_0)**2 

Re = a/sqrt(1 - e2*sin(phi_0)**2) 
iterate_count = 0
print('initial coarse lambda, phi, height: ', lambd*180/pi, phi_0*180/pi, h_0)
while stop_condition > 0.001**2:
    iterate_count +=1

    #calculate temp derivatives
    Re = a/sqrt(1 - e2*sin(phi_0)**2) 
    dRe_dphi = - (a * e2 * sin(phi_0) * cos(phi_0))/((1 - e2 * sin(phi_0)**2)**(3/2)) 
    df1_dphi = - dRe_dphi * cos(phi_0) +(Re + h_0)*sin(phi_0) 
    df2_dphi = - (1 - e2) * dRe_dphi * sin(phi_0) - ((1 - e2) * Re + h_0) * cos(phi_0) 
    df1_dh   = - cos(phi_0) 
    df2_dh   = - sin(phi_0) 
    
    f1 = sqrt(eta_1**2 + eta_2**2) - (Re + h_0) * cos(phi_0) 
    f2 = eta_3 - ((1 - e2) * Re + h_0)*sin(phi_0) 
    # solve the equation
    eq_coeff = df1_dphi * df2_dh - df2_dphi * df1_dh 
    phi_1 = phi_0 - (1/eq_coeff) * (f1 * df2_dh - f2 * df1_dh) 
    h_1 = h_0 - (1/eq_coeff) * (- f1 * df2_dphi + f2 * df1_dphi) 
    
    delta_phi = phi_1 - phi_0 
    delta_h = h_1 - h_0 
    stop_condition = delta_phi**2 * a**2 + delta_h**2 
    print(iterate_count,'lambda (degrees), phi (degrees), height (meters): ', lambd*180/pi, phi_1*180/pi, h_1)
    phi_0 = phi_1 
    h_0 = h_1 
print('\nSTOP\n')
print('Final results: Geographic Frame-of-Reference\n',
      'number of iterations:',iterate_count,'\n lambda:',lambd*180/pi,'degrees\n',
      'phi:   ',phi_1*180/pi, 'degrees\n','height:',h_1,'meters')

print('Final results: Geodetic\n',
      'number of iterations:',iterate_count,'\n lambda:',lambd*180/pi,'degrees\n',
      'phi0:  ',phi0*180/pi, 'degrees\n','height:',h_1,'meters')

0.00669436619
initial coarse lambda, phi, height:  43.9636650190588 56.336859891872116 400.28741593852465
1 lambda (degrees), phi (degrees), height (meters):  43.9636650190588 56.33721761434298 387.21258556662553
2 lambda (degrees), phi (degrees), height (meters):  43.9636650190588 56.337217621949414 386.96616586996197
3 lambda (degrees), phi (degrees), height (meters):  43.9636650190588 56.33721762194956 386.96616062782533

STOP

Final results: Geographic Frame-of-Reference
 number of iterations: 3 
 lambda: 43.9636650190588 degrees
 phi:    56.33721762194956 degrees
 height: 386.96616062782533 meters
Final results: Geodetic
 number of iterations: 3 
 lambda: 43.9636650190588 degrees
 phi0:   56.1594476762467 degrees
 height: 386.96616062782533 meters


## 2. 
Пусть заданы географические координаты — долгота, широта, высота:
$$\lambda = 0^◦ 16' 46.6381'' ,
\varphi = 51^◦ 33' 21.1362'' ,
h = 428.923 м.$$
Вычислите для этих координат значение вектора удельной силы тяготения двумя
способами — через формулу Гельмерта и модель ГЛОНАСС. Сравните результаты
в осях географического трехгранника.

In [3]:
# problem variables and constants using the GLONASS model 
lam = (0 + (16/60) + (46.6381/3600))*pi/180  # decimal representation in radians
phi = (51 + (33/60) + (21.1362/3600))*pi/180
h = 428.923
u = 7.2921158553 * 10 ** (-5)


# Gelmert model


om_02 = 1.543 * 10 ** (-6)
print('initial parameters:', lam, phi, h)
g = 9.78030*(1 + 0.005302*sin(phi)**2 - 0.000007*sin(2*phi)**2) - 0.00014 - 2*om_02 * h
gelmert_x0 = np.matrix([0, 
          u**2 * (Re + h) * sin(phi)*cos(phi),
          -g - u**2 * (Re + h) * cos(phi)]).T
print('gelmert_x0 and absolute value:\n',gelmert_x0, sqrt(gelmert_x0[0]**2+gelmert_x0[1]**2+gelmert_x0[2]**2), 
      'm/s^2')


# GLONASS model


Re = a/sqrt(1 - e2*sin(phi)**2) 
eta_1 = (Re + h)*cos(phi)*cos(lam)
eta_2 = (Re + h)*cos(phi)*sin(lam)
eta_3 = (Re + h)*sin(phi) - Re * e2 * sin(phi)
mu = 398600.44 * 10**9
a = 6378136
e2 = 6.69436619 * 10**(-3)
C20 = - 1082.6257 * 10**(-6)
r = sqrt(eta_1**2+eta_2**2+eta_3**2)


g_eta1 = -mu/(r**3)*(1+1.5*C20*(a**2)/(r**2)*(5*eta_3**2/(r**2)-1))*eta_1 
g_eta2 = -mu/(r**3)*(1+1.5*C20*(a**2)/(r**2)*(5*eta_3**2/(r**2)-1))*eta_2
g_eta3 = -mu/(r**3)*(1+1.5*C20*(a**2)/(r**2)*(5*eta_3**2/(r**2)-1))*eta_3
glonass_eta = np.matrix([g_eta1, g_eta2, g_eta3]).T
print('glonass_eta and absolute value:\n', glonass_eta, sqrt(g_eta1**2+g_eta2**2+g_eta3**2), 'm/s^2')

# transfer to the geografic frame

A_x0eta = np.matrix([[-sin(lam)          ,  cos(lam)          , 0],
                     [-cos(lam)*sin(phi) , -sin(lam)*sin(phi) , cos(phi)],
                     [ cos(lam)*cos(phi) ,  sin(lam)*cos(phi) , sin(phi)]])
glonass_x0 = A_x0eta @ glonass_eta  # matrix product is denoted by @
print('glonass_x0 and absolute value:\n',glonass_x0, 'm/s^2')


print('delta_g: gelmert_x0 - glonass_x0\n', gelmert_x0 - glonass_x0, 'm/s^2')

initial parameters: 0.004880319228061092 0.8998197005923435 428.923
gelmert_x0 and absolute value:
 [[ 0.        ]
 [ 0.01655529]
 [-9.83171829]] 9.831732229384796 m/s^2
glonass_eta and absolute value:
 [[-6.12090208]
 [-0.02987219]
 [-7.6589277 ]] 9.804361687693426 m/s^2
glonass_x0 and absolute value:
 [[ 3.46944695e-18]
 [ 3.20908944e-02]
 [-9.80430917e+00]] m/s^2
delta_g: gelmert_x0 - glonass_x0
 [[-3.46944695e-18]
 [-1.55355996e-02]
 [-2.74091221e-02]] m/s^2


## 3. 
Вычислите с точностью до $e^6$ модули радиусов кривизны $R_E, R_N$ . Сравните приближенные значения с точными, используя географические координаты из предыдущей
задачи.

In [4]:
phi = 51 + (33/60) + (21.1362/3600)
a = 6378136
e2 = 6.69436619 * 10**(-3)
# precise Re and Rn:
Re = a/sqrt(1 - e2*sin(phi)**2)
Rn = (a*(1-e2))/(1 - e2*sin(phi)**2)**1.5
# Taylor series up to e^6 for Re and Rn :
Re_taylor = a*(1 + (1/2) * e2 * sin(phi)**2 + (3/8)*e2**2 * sin(phi)**4 + (15/48)*e2**3 * sin(phi)**6)
Rn_taylor = a*(1+e2*(1.5*sin(phi)**2-1) +
               (3/2)*e2**2*sin(phi)**2*((5/4)*sin(phi)**2 - 1) +
               (5/8)*e2**3 * sin(phi)**4*((7/2)*sin(phi)**2-3) )
print('approximate Rn accurate up to e^6:',Rn_taylor, 'm',
      '\napproximate Re accurate up to e^6:',Re_taylor,'m')
print('precise Rn:',Rn, 'm','\nprecise Re:',Re, 'm')
print('delta Re:  ',Re-Re_taylor, 'm','\ndelta Rn:  ',Rn-Rn_taylor, 'm')

approximate Rn accurate up to e^6: 6394640.8463787725 m 
approximate Re accurate up to e^6: 6397941.572579555 m
precise Rn: 6394640.847238055 m 
precise Re: 6397941.575140457 m
delta Re:   0.002560901455581188 m 
delta Rn:   0.000859282910823822 m


## 4.
Вычислите кватернион ориентации приборного трехгранника $Ms$ относительно гринвичского трехгранника $Oη$.

### Решение.
Имеем кватернионы ориентации гринвичского трехгранника $M\eta $ относительно географического трехгранника $Mx^0$
и кватернион ориентации географического трехгранника $Mx^0$ относительно приборного $Ms$:

\begin{eqnarray*}
P^{  \eta x^0}=\frac{1}{2}
 \left(
 \begin{array}{c}
 \phantom{-} \cos \frac{\lambda + \varphi}{2} - \sin \frac{ \lambda - \varphi}{2} \cr
 -\cos \frac{\lambda - \varphi}{2} + \sin \frac{ \lambda + \varphi}{2} \cr
 -\cos \frac{\lambda + \varphi}{2} - \sin \frac{ \lambda - \varphi}{2} \cr
 -\cos \frac{\lambda - \varphi}{2} - \sin \frac{ \lambda + \varphi}{2}
\end{array}
\right),
\end{eqnarray*}




\begin{eqnarray*}
P^{x^0s}= \left( \begin{array}{c}
\phantom{-} \cos \frac{\psi}{2} \cos \frac{\vartheta}{2}  \cos \frac{\gamma}{2} + \sin \frac{\psi}{2} \sin \frac{\vartheta}{2}  \sin \frac{\gamma}{2} \cr
-\cos \frac{\psi}{2} \sin \frac{\vartheta}{2}  \cos \frac{\gamma}{2} - \sin \frac{\psi}{2} \cos \frac{\vartheta}{2}  \sin \frac{\gamma}{2} \cr
- \cos \frac{\psi}{2} \cos \frac{\vartheta}{2}  \sin \frac{\gamma}{2} + \sin \frac{\psi}{2} \sin \frac{\vartheta}{2}  \cos \frac{\gamma}{2} \cr
\phantom{-} \sin \frac{\psi}{2} \cos \frac{\vartheta}{2}  \cos \frac{\gamma}{2} - \cos \frac{\psi}{2} \sin \frac{\vartheta}{2}  \sin \frac{\gamma}{2}
\end{array} \right).
\end{eqnarray*}

Результирующий поворот вычисляется по формуле:
\begin{eqnarray*}
r_0 &=& p_0 q_0 - p_1 q_1 - p_2 q_2 - p_3 q_3 \nonumber \\
r_1 &=& q_0 p_1 + p_0 q_1 - p_3 q_2 + p_2 q_3 \nonumber \\
r_2 &=& q_0 p_2 + p_0 q_2 + p_3 q_1  - p_1 q_3 \nonumber \\
r_3 &=& q_0 p_3 + p_0 q_3 - p_2 q_1 + p_1 q_2,
\end{eqnarray*}
где параметры $p$ отвечают первому повороту, параметры $q$ - второму, а $r$ - результирующему повороту. Для получения верного ответа нужно найти сопряжённый кватернион
$\widetilde{P}^{\eta s}= (r_0, -r_1, -r_2, -r_3)^T$, отвечающий обратному повороту:
\begin{eqnarray*}
P^{s \eta} = \widetilde{P}^{\eta s}.
\end{eqnarray*}

Выражение вышло очень громоздким и трудносокращаемым. Чтобы перепроверить собственные выкладки, подключим библиотеку вычисления символьных математических выражений:

In [ ]:
from sympy import * # allows using symbolic mathematical equations
init_printing(use_unicode=True)

# add our angles as symbols
phi = symbols('varphi')
lam = symbols('lambda')
psi = symbols('psi')
the = symbols('vartheta')
gam = symbols('gamma')
# 'the' stands for theta for sake of readability like 'gam' for gamma and 'lam' for lambda

p0 =   (cos((lam+phi)/2) - sin((lam-phi)/2))/2
p1 = - (cos((lam-phi)/2) + sin((lam+phi)/2))/2
p2 = - (cos((lam+phi)/2) - sin((lam-phi)/2))/2
p3 = - (cos((lam-phi)/2) - sin((lam+phi)/2))/2

q0 =   cos(psi/2)*cos(the/2)*cos(gam/2) + sin(psi/2)*sin(the/2)*sin(gam/2)
q1 = - cos(psi/2)*sin(the/2)*cos(gam/2) - sin(psi/2)*cos(the/2)*sin(gam/2)
q2 = - cos(psi/2)*cos(the/2)*sin(gam/2) + sin(psi/2)*sin(the/2)*cos(gam/2)
q3 =   sin(psi/2)*cos(the/2)*cos(gam/2) - cos(psi/2)*sin(the/2)*sin(gam/2)

r0 = p0*q0 - p1*q1 - p2*q2 - p3*q3
r1 = q0*p1 + p0*q1 - p3*q2 + p2*q3
r2 = q0*p2 + p0*q2 + p3*q1 - p1*q3
r3 = q0*p3 + p0*q3 - p2*q1 + p1*q2

# preparation to include equation in LaTeX
print(latex(trigsimp(r0, fold_func_brackets = True)),'\n')
print(latex(trigsimp(-r1)),'\n')
print(latex(trigsimp(-r2)),'\n')
print(latex(trigsimp(-r3)),'\n')

Таким образом, результирующий кватернион ориентации приборного трехгранника $Ms$ относительно гринвичского трехгранника $Oη$ имеет такой вид (покоординатно):

\begin{align}
r_0 = \left(\sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\psi}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} + \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)}\right) \sin{\left(\frac{\varphi}{2} + \frac{\pi}{4} \right)} \cos{\left(\frac{\lambda}{2} + \frac{\pi}{4} \right)} - \left(\sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)} + \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)}\right) \sin{\left(\frac{\lambda}{2} + \frac{\pi}{4} \right)} \sin{\left(\frac{\varphi}{2} + \frac{\pi}{4} \right)} - \left(\sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} - \sin{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)}\right) \cos{\left(\frac{2 \lambda + \pi}{4} \right)} \cos{\left(\frac{2 \varphi + \pi}{4} \right)} - \left(\sin{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)} - \sin{\left(\frac{\psi}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)}\right) \sin{\left(\frac{2 \varphi + \pi}{4} \right)} \cos{\left(\frac{2 \lambda + \pi}{4} \right)} 
 \nonumber \\
\end{align}
\begin{align}
r_1 = \left(\sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\psi}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} + \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)}\right) \sin{\left(\frac{\lambda}{2} + \frac{\pi}{4} \right)} \sin{\left(\frac{\varphi}{2} + \frac{\pi}{4} \right)} + \left(\sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)} + \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)}\right) \sin{\left(\frac{\varphi}{2} + \frac{\pi}{4} \right)} \cos{\left(\frac{\lambda}{2} + \frac{\pi}{4} \right)} - \left(\sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} - \sin{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)}\right) \sin{\left(\frac{2 \varphi + \pi}{4} \right)} \cos{\left(\frac{2 \lambda + \pi}{4} \right)} + \left(\sin{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)} - \sin{\left(\frac{\psi}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)}\right) \cos{\left(\frac{2 \lambda + \pi}{4} \right)} \cos{\left(\frac{2 \varphi + \pi}{4} \right)} 
  \nonumber \\
\end{align}
\begin{align}
r_2 = - \left(\sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\psi}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} + \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)}\right) \left(\frac{\sin{\left(\frac{\lambda - \varphi}{2} \right)}}{2} - \frac{\cos{\left(\frac{\lambda + \varphi}{2} \right)}}{2}\right) + \left(\sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)} + \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)}\right) \left(\frac{\sin{\left(\frac{\lambda + \varphi}{2} \right)}}{2} - \frac{\cos{\left(\frac{\lambda - \varphi}{2} \right)}}{2}\right) - \left(- \sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} + \sin{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)}\right) \sin{\left(\frac{\lambda}{2} + \frac{\pi}{4} \right)} \sin{\left(\frac{\varphi}{2} + \frac{\pi}{4} \right)} - \left(- \sin{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)} + \sin{\left(\frac{\psi}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)}\right) \sin{\left(\frac{\varphi}{2} + \frac{\pi}{4} \right)} \cos{\left(\frac{\lambda}{2} + \frac{\pi}{4} \right)}    \nonumber \\
\end{align}
\begin{align}
r_3 = - \left(\sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\psi}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} + \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)}\right) \left(\frac{\sin{\left(\frac{\lambda + \varphi}{2} \right)}}{2} - \frac{\cos{\left(\frac{\lambda - \varphi}{2} \right)}}{2}\right) - \left(\sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)} + \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)}\right) \left(\frac{\sin{\left(\frac{\lambda - \varphi}{2} \right)}}{2} - \frac{\cos{\left(\frac{\lambda + \varphi}{2} \right)}}{2}\right) - \left(- \sin{\left(\frac{\gamma}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} + \sin{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)}\right) \sin{\left(\frac{\varphi}{2} + \frac{\pi}{4} \right)} \cos{\left(\frac{\lambda}{2} + \frac{\pi}{4} \right)} + \left(- \sin{\left(\frac{\gamma}{2} \right)} \cos{\left(\frac{\psi}{2} \right)} \cos{\left(\frac{\vartheta}{2} \right)} + \sin{\left(\frac{\psi}{2} \right)} \sin{\left(\frac{\vartheta}{2} \right)} \cos{\left(\frac{\gamma}{2} \right)}\right) \sin{\left(\frac{\lambda}{2} + \frac{\pi}{4} \right)} \sin{\left(\frac{\varphi}{2} + \frac{\pi}{4} \right)} 
\end{align}

Я честно пытался придумать какие-то замены, существенно упрощающие вид выражений, но не смог.








## 5. 
Приведите расчетные формулы (по Сэвиджу) для двухшагового метода в задаче
ориентации (в скалярной форме).

### Решение
Имеем расчётную формулу (по Сэвиджу) в таком виде:
\begin{eqnarray*}
\gamma_y = \sum\limits_{k=1}^n \gamma_{y_k} -
\sum\limits_{k=2}^n\left[\frac12\left(\sum\limits_{i=1}^{k-1}\widehat{\gamma}_{y_i} +
\frac16\widehat{\gamma}_{y_{k-1}}\right)\gamma_{y_k}\right].
\end{eqnarray*}
При $n = 2$ можем раскрыть знаки сумм следующим образом:
\begin{eqnarray*}
\gamma_y = \gamma_{y_1} + \gamma_{y_2} -
 \frac12\left(\widehat{\gamma}_{y_1} +
\frac16\widehat{\gamma}_{y_1}\right)\gamma_{y_2}.
\end{eqnarray*}

Раскрывая далее, получим:
\begin{eqnarray*}
\gamma_y = \gamma_{y_1} + \gamma_{y_2} -
 \frac{7}{12}\widehat{\gamma}_{y_1}\gamma_{y_2}.
\end{eqnarray*}
Чтобы не запутаться в индексах, однообразно обозначающих и момент времени (номер измерения), и номер координаты, будем обозначать номер измерения в скобках, а номер координаты -- нижним индексом, а именно так: $\gamma_{y1}(2)$ -- значение первой координаты вектора $\gamma$ в отсчёт времени 2.
Тогда получим:
\begin{eqnarray*}
\gamma_{y1} &=& \gamma_{y1}(1)+\gamma_{y1}(2)-\frac{7}{12}\left(\phantom{-}\gamma_{y3}(1)\gamma_{y2}(2)-\gamma_{y2}(1)\gamma_{y3}(2) \right), \\
\gamma_{y2} &=& \gamma_{y2}(1)+\gamma_{y2}(2)-\frac{7}{12}\left(-\gamma_{y3}(1)\gamma_{y2}(2)+\gamma_{y1}(1)\gamma_{y3}(2) \right), \\
\gamma_{y3} &=& \gamma_{y3}(1)+\gamma_{y3}(2)-\frac{7}{12}\left(\phantom{-}\gamma_{y2}(1)\gamma_{y1}(2)-\gamma_{y1}(1)\gamma_{y2}(2) \right). \\
\end{eqnarray*}